# A09. Contest Guides
This creates contest guides used to link contest and game identifiers
- Type: Data
- Run Frequency: Pre-contest, refresh of yesterday
- Sources:
    - MLB Stats API
    - Daily Fantasy Fuel Slates
    - RotoWire Slates
    - DraftKings contest history
- Dates:
    - Created: 3/30/2024
    - Updated: 4/21/2024

### Imports

In [2]:
if not hasattr(sys.modules['__main__'], '__file__'):
    %run "C:\Users\james\Documents\MLB\Code\U1. Imports.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U2. Utilities.ipynb"
    %run "C:\Users\james\Documents\MLB\Code\U3. Classes.ipynb"
    print("Imports executed")

Imports executed


### Games

In [8]:
if not hasattr(sys.modules['__main__'], '__file__'):
    # Set date range 
    start_date = todaysdate
    end_date = todaysdate
    game_df = create_games(start_date, end_date, team_dict)

### Functions

##### Slate Name

In [13]:
# Identify DraftKings slate name
def pick_slate(Name):
    if "(Early)" in Name:
        slate = "Early"
    elif "(Late Night)" in Name:
        slate = "Late Night"
    elif "Night" in Name:
        slate = "Night"
    elif "Afternoon" in Name:
        slate = "Afternoon"
    else:
        slate = "All"

    
    return slate

##### Historic

Building contest guides from Contests.csv

In [17]:
def historic_contest_guide(all_games_df, contestKey):
    # Selected contest
    selected_contest = contests.query(f'contestKey == {contestKey}').reset_index(drop=True)

    # Identify slate name (Early, Afternoon, Night, Late Night, All)
    selected_contest['slate'] = selected_contest['name'].apply(pick_slate)
    
    # Extract date and slate name
    date = selected_contest['date'][0]
    slate = selected_contest['slate'][0]
    
    # Identify slate IDs
    # DFF
    # Can be missing, might need to use date-based slates
    try:
        # Read in slates
        dff_slates = pd.read_csv(os.path.join(baseball_path, "A07. Projections", "1. DFF", "1. Slates", f"DFF Slates {int(date)}.csv"))

        # Identify slate ID
        slateID = dff_slates.loc[(dff_slates['date'] == date) & (dff_slates['Slate Type'] == slate), 'URL'].values[0]

        # Make it an integer, if possible
        try:
            slateID = int(slateID)
        except:
            pass
    except:
        slateID = np.nan

        
    # Assign
    selected_contest['dff_slate'] = slateID
    
    # RotoWire
    # Shouldn't really be missing, but allow for it.
    try:
        # Read in Roto slates
        roto_slates = pd.read_csv(os.path.join(baseball_path, "A07. Projections", "2. RotoWire", "1. Slates", f"RotoWire Slates {int(date)}.csv"))

        # Identify slate ID
        slateID = roto_slates.loc[(roto_slates['date'] == date) & (roto_slates['name'] == slate), 'slateID'].values[0]
    except:
        slateID = np.nan

    # Assign
    selected_contest['roto_slate'] = slateID
    
    
    # Determine matchups
    # Extract draftGroupId
    draftGroupId = selected_contest['draftGroupId'][0]
    # Read in draftable players
    draftables = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f'Draftables {int(draftGroupId)}.csv'), encoding='iso-8859-1')
    # Identify matchups
    slate_matchups = draftables[['Game Info', 'draftGroupId']].drop_duplicates('Game Info', keep='first')

    # Merge matchups onto contest
    selected_contest = selected_contest.merge(slate_matchups, on='draftGroupId', how='inner')

    # Identify away team
    selected_contest['DKTEAM'] = selected_contest['Game Info'].str.split("@", expand=True)[0]

    # Merge on BBREFTEAM ID for merging with all_game_df
    selected_contest = selected_contest.merge(team_map[['DKTEAM', 'BBREFTEAM']], on='DKTEAM', how='left')
    
    # Merge with all_games_df for more game information (game_id, in particular)
    selected_contest = selected_contest.merge(all_games_df[['date', 'away_team', 'game_id', 'game_type', 'status', 'game_num', 'away_score', 'home_score']], left_on=['BBREFTEAM', 'date'], right_on=['away_team', 'date'], how='inner')

    # Handle doubleheaders
    # If there are multiple instances of a team, we want to keep the first if it's an early slate or the second if it's a later slate.
    if slate in ['Early', 'Afternoon']:
        selected_contest.drop_duplicates('DKTEAM', keep='first', inplace=True)
    else:
        selected_contest.drop_duplicates('DKTEAM', keep='last', inplace=True)
        
    selected_contest['date'] = selected_contest['date'].astype('int')
    selected_contest['draftGroupId'] = selected_contest['draftGroupId'].astype('int')

    
    return selected_contest[['contestKey', 'draftGroupId', 'date', 'entryFee', 'name', 'slate', 'dff_slate', 'roto_slate', 'Game Info', 'game_id', 'game_type', 'game_num', 'away_score', 'home_score']]

##### Present

In [135]:
def contest_guide(game_df, subset_df, contestKey):  
    # Copy game_df 
    game_df_copy = game_df.copy()
    # Convert date to int
    game_df_copy['date'] = game_df_copy['date'].astype(int)

    # Selected contest
    selected_contest = subset_df.query(f'contestKey == {contestKey}').reset_index(drop=True)

    # Identify slate name (Early, Afternoon, Night, Late Night, All)
    selected_contest['slate'] = selected_contest['Name'].apply(pick_slate)   
    
    # Extract date and slate name
    date = selected_contest['date'][0]
    slate = selected_contest['slate'][0]


    # Identify slate IDs
    # DFF
    # Can be missing, might need to use date-based slates
    try:
        # Read in slates
        dff_slates = pd.read_csv(os.path.join(baseball_path, "A07. Projections", "1. DFF", "1. Slates", f"DFF Slates {int(date)}.csv"))

        # Identify slate ID
        slateID = dff_slates.loc[(dff_slates['date'] == date) & (dff_slates['Slate Type'] == slate), 'URL'].values[0]

        # Make it an integer, if possible
        try:
            slateID = int(slateID)
        except:
            pass
    except:
        slateID = np.nan

        
    # Assign DFF slate
    selected_contest['dff_slate'] = slateID
    
    # RotoWire
    # Shouldn't really be missing, but allow for it.
    try:
        # Read in Roto slates
        roto_slates = pd.read_csv(os.path.join(baseball_path, "A07. Projections", "2. RotoWire", "1. Slates", f"RotoWire Slates {int(date)}.csv"))

        # Identify slate ID
        slateID = roto_slates.loc[(roto_slates['date'] == date) & (roto_slates['name'] == slate), 'slateID'].values[0]
    except:
        slateID = np.nan

    # Assign RotoWire slate
    selected_contest['roto_slate'] = slateID
    
    # Determine matchups
    # Extract draftGroupId
    draftGroupId = selected_contest['draftGroupId'][0]
    # Read in draftable players
    draftables = pd.read_csv(os.path.join(baseball_path, "A01. DraftKings", "2. Draftables", f'Draftables {int(draftGroupId)}.csv'), encoding='iso-8859-1')
    # Identify matchups
    slate_matchups = draftables[['Game Info', 'draftGroupId']].drop_duplicates('Game Info', keep='first')

    # Merge matchups onto contest
    selected_contest = selected_contest.merge(slate_matchups, on='draftGroupId', how='inner')
    
    # Identify away team
    selected_contest['DKTEAM'] = selected_contest['Game Info'].str.split("@", expand=True)[0]
    selected_contest['BBREFTEAM'] = selected_contest['DKTEAM'].map(team_dict)
    
    # Merge with game_df_copy for more game information (game_id, in particular)
    selected_contest = selected_contest.merge(game_df_copy[['date', 'away_team', 'game_id', 'game_type', 'status', 'game_num', 'away_score', 'home_score']], left_on=['BBREFTEAM', 'date'], right_on=['away_team', 'date'], how='inner')

    # Handle doubleheaders
    # If there are multiple instances of a team, we want to keep the first if it's an early slate or the second if it's a later slate.
    if slate in ['Early', 'Afternoon']:
        selected_contest.drop_duplicates('DKTEAM', keep='first', inplace=True)
    else:
        selected_contest.drop_duplicates('DKTEAM', keep='last', inplace=True)
        
    selected_contest['date'] = selected_contest['date'].astype('int')
    selected_contest['draftGroupId'] = selected_contest['draftGroupId'].astype('int')

    # Rename to match previous naming conventions from when guides were built using Contest history, not subset_df
    selected_contest.rename(columns={'Name':'name', 'Entry Fee': 'entryFee'}, inplace=True)

    
    return selected_contest[['contestKey', 'draftGroupId', 'date', 'entryFee', 'name', 'slate', 'dff_slate', 'roto_slate', 'Game Info', 'game_id', 'game_type', 'game_num', 'away_score', 'home_score']]

### Run

In [145]:
for date in game_df['date'].unique():
    print(date)
    # Read in contest subset
    subset_df = pd.read_csv(os.path.join(baseball_path, 'A01. DraftKings', '7. Subsets', f'Subset {date}.csv'))
    
    # Loop over contestKeys
    for contestKey in subset_df['contestKey'].reset_index(drop=True):
        print(contestKey)
        try:
            guide = contest_guide(game_df, subset_df=subset_df, contestKey=contestKey)
            if not guide.empty:
                guide.to_csv(os.path.join(baseball_path, "A09. Contest Guides", f"Contest Guide {contestKey}.csv"), index=False)
            else:
                print(f"Contest Guide {contestKey} is empty.")
        except FileNotFoundError as e:
            print(f"Draftables {contestKey}.csv not found.")

20250327
175193358
SL                        Game Info  draftGroupId
0  ATH@SEA 03/27/2025 10:10PM ET        123896
1  CHC@ARI 03/27/2025 10:10PM ET        123896
SC                                           Name Cash Prize  Entry Fee  \
0  MLB $15K Four-Seamer [20 Entry Max] (Night)    15000.0        4.0   
1  MLB $15K Four-Seamer [20 Entry Max] (Night)    15000.0        4.0   

   contestKey  draftGroupId          contestDate  contestTime gameType  \
0   175193358        123896  2025-03-27 22:10:00  Thu 10:10PM  Classic   
1   175193358        123896  2025-03-27 22:10:00  Thu 10:10PM  Classic   

       date   date_dash  slate dff_slate  roto_slate  \
0  20250327  2025-03-27  Night     1E3F8       19504   
1  20250327  2025-03-27  Night     1E3F8       19504   

                       Game Info DKTEAM  
0  ATH@SEA 03/27/2025 10:10PM ET    ATH  
1  CHC@ARI 03/27/2025 10:10PM ET    CHC  
   contestKey  draftGroupId      date  entryFee  \
0   175193358        123896  20250327       4.0 

In [143]:
game_df

,game_id,game_datetime,game_date,game_type,status,away_name,home_name,away_id,home_id,doubleheader,game_num,home_probable_pitcher,away_probable_pitcher,away_score,home_score,current_inning,inning_state,venue_id,venue_name,winning_team,losing_team,winning_pitcher,losing_pitcher,save_pitcher,date,year,away_team,home_team
0,778557,2025-03-27T19:05:00Z,2025-03-27,R,Final,Milwaukee Brewers,New York Yankees,158,147,N,1,Carlos Rodón,Freddy Peralta,2,4,9,Top,3313,Yankee Stadium,New York Yankees,Milwaukee Brewers,Carlos Rodón,Freddy Peralta,Devin Williams,20250327,2025,MIL,NYY
1,778556,2025-03-27T19:07:00Z,2025-03-27,R,Final,Baltimore Orioles,Toronto Blue Jays,110,141,N,1,José Berríos,Zach Eflin,12,2,9,Bottom,14,Rogers Centre,Baltimore Orioles,Toronto Blue Jays,Zach Eflin,José Berríos,None,20250327,2025,BAL,TOR
2,778553,2025-03-27T20:05:00Z,2025-03-27,R,Final,Boston Red Sox,Texas Rangers,111,140,N,1,Nathan Eovaldi,Garrett Crochet,5,2,9,Bottom,5325,Globe Life Field,Boston Red Sox,Texas Rangers,Aroldis Chapman,Luke Jackson,Justin Slaten,20250327,2025,BOS,TEX
3,778555,2025-03-27T20:05:00Z,2025-03-27,R,Final,Philadelphia Phillies,Washington Nationals,143,120,N,1,MacKenzie Gore,Zack Wheeler,7,3,10,Bottom,3309,Nationals Park,Philadelphia Phillies,Washington Nationals,José Alvarado,Colin Poche,None,20250327,2025,PHI,WSN
4,778558,2025-03-27T20:10:00Z,2025-03-27,R,Final,Cleveland Guardians,Kansas City Royals,114,118,N,1,Cole Ragans,Ben Lively,7,4,10,Bottom,7,Kauffman Stadium,Cleveland Guardians,Kansas City Royals,Emmanuel Clase,Sam Long,Paul Sewald,20250327,2025,CLE,KCR
5,778559,2025-03-27T20:10:00Z,2025-03-27,R,Final,New York Mets,Houston Astros,121,117,N,1,Framber Valdez,Clay Holmes,1,3,9,Top,2392,Daikin Park,Houston Astros,New York Mets,Framber Valdez,Clay Holmes,Josh Hader,20250327,2025,NYM,HOU
6,778561,2025-03-27T20:10:00Z,2025-03-27,R,Final,San Francisco Giants,Cincinnati Reds,137,113,N,1,Hunter Greene,Logan Webb,6,4,9,Bottom,2602,Great American Ball Park,San Francisco Giants,Cincinnati Reds,Tyler Rogers,Ian Gibaut,Ryan Walker,20250327,2025,SFG,CIN
7,778545,2025-03-27T20:10:00Z,2025-03-27,R,Final,Atlanta Braves,San Diego Padres,144,135,N,1,Michael King,Chris Sale,4,7,9,Top,2680,Petco Park,San Diego Padres,Atlanta Braves,Wandy Peralta,Héctor Neris,Robert Suarez,20250327,2025,ATL,SDP
8,778554,2025-03-27T20:10:00Z,2025-03-27,R,Final,Los Angeles Angels,Chicago White Sox,108,145,N,1,Sean Burke,Yusei Kikuchi,1,8,9,Top,4,Rate Field,Chicago White Sox,Los Angeles Angels,Sean Burke,Yusei Kikuchi,None,20250327,2025,LAA,CHW
9,778562,2025-03-27T20:10:00Z,2025-03-27,R,Final,Pittsburgh Pirates,Miami Marlins,134,146,N,1,Sandy Alcantara,Paul Skenes,4,5,9,Bottom,4169,loanDepot park,Miami Marlins,Pittsburgh Pirates,Jesus Tinoco,David Bednar,None,20250327,2025,PIT,MIA


In [31]:
subset_df.head()

,Name,Cash Prize,Entry Fee,contestKey,draftGroupId,contestDate,contestTime,gameType,date,date_dash
0,MLB $30K Relay Throw [$10K to 1st] (Early),30000.0,15.0,175193307,124555,2025-03-27 15:05:00,Thu 3:05PM,Classic,20250327,2025-03-27
1,MLB $3K Solo Shot (Early),3000.0,1.0,175193310,124555,2025-03-27 15:05:00,Thu 3:05PM,Classic,20250327,2025-03-27
2,MLB $1.5K Four-Seamer (Early),1500.0,4.0,175193308,124555,2025-03-27 15:05:00,Thu 3:05PM,Classic,20250327,2025-03-27
3,MLB $1K Pinch Hit [Single Entry] (Early),1000.0,50.0,175193306,124555,2025-03-27 15:05:00,Thu 3:05PM,Classic,20250327,2025-03-27
4,MLB $1K Chin Music [Single Entry] (Early),1000.0,5.0,175193313,124555,2025-03-27 15:05:00,Thu 3:05PM,Classic,20250327,2025-03-27
